## Unity Catalog – ADLS Connection Steps

### Step 1: Grant Azure Storage Permissions
- Go to the **Storage Account** in Azure.
- Open **Access Control (IAM)**.
- Assign the **Storage Blob Data Contributor** role to the  
  **unity-catalog-access-connector** (Managed Identity).

This allows Databricks (via Unity Catalog) to access the ADLS container.


### Step 2: Create External Location and Storage Credential
- In **Databricks → Catalog Explorer → External Location**:
  - Create a **Storage Credential** using **Managed Identity**.
  - Provide the **Access Connector Resource ID** of the  
    `unity-catalog-access-connector`.
- Create an **External Location** pointing to your ADLS path  
  (for example: `abfss://<container>@<storage-account>.dfs.core.windows.net/`).
- Attach the newly created **Storage Credential** to this External Location.

This links Unity Catalog securely to your ADLS storage.


### Step 3: Create Schema and Volume
- Create a **Schema** under your Unity Catalog.
- Create a **Volume** on top of the External Location.
- Grant required permissions (`READ`, `WRITE`) on the Volume to users or groups.

From now on, all data access happens **through the Volume**, not via direct `abfss://` paths.



In [0]:
%sql
SHOW CATALOGS;

catalog
sales_databricks_workspace
samples
system


In [0]:
%sql
USE CATALOG sales_databricks_workspace;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS sales_databricks_workspace.raw;



In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS sales_databricks_workspace.bronze;

In [0]:
%sql
show schemas

databaseName
bronze
default
information_schema
raw


In [0]:
%sql
CREATE EXTERNAL VOLUME sales_databricks_workspace.raw.lakehouse_retail_vol
LOCATION 'abfss://sales-data@saleslakestorage.dfs.core.windows.net/raw'


In [0]:
df = spark.read.option("header", "true").csv(
    "/Volumes/sales_databricks_workspace/raw/lakehouse_retail_vol/sales/3a_superstore/customers/Customers_ENG.csv")
df.display()


USERID;USERNAME_;NAMESURNAME;STATUS_;USERGENDER;USERBIRTHDATE;REGION;CITY;TOWN;DISTRICT;ADDRESSTEXT
1;atilla.keles24086@outlookerlab.com;Atilla Keleş;1;M;1965-03-29;Ic Anadolu;Eskisehir;TEPEBASI;FATIH MAH.;FATIH MAH. SEHIT AHMET KOC SOKAK 26220 TEPEBASI/ESKISEHIR
2;zeynep.seyma.altun98278@outlookerlab.com;Zeynep Şeyma Altun;1;F;1980-09-23;Guneydogu Anadolu;Mardin;KIZILTEPE;ARIKLI KOYU;ARIKLI KOYU MAH. KOYUN KENDISI 47400 KIZILTEPE/MARDIN
3;aysun.aydin80939@inboxplus.com;Aysun Aydın;1;F;1961-06-16;Marmara;Istanbul;CATALCA;FERHATPASA MAH.;FERHATPASA MAH. NAR SOKAK 34540 CATALCA/ISTANBUL
4;lara.cinar80899@outlookerlab.com;Lara Çınar;1;F;1955-12-21;Karadeniz;Rize;FINDIKLI;CENNET KOYU;CENNET KOYU MAH. DUZ CADDESI 53740 FINDIKLI/RIZE
5;gonul.seda.cetin85529@gmaileo.com;Gönül Seda Çetin;1;F;1986-08-09;Ege;Izmir;KIRAZ;CERITLER KOYU;CERITLER KOYU MAH. SULUKLU MEVKI 35890 KIRAZ/IZMIR
6;yasemin.celik61459@inboxplus.com;Yasemin Çelik;1;F;1970-02-06;Marmara;Istanbul;MALTEPE;IDEALTEPE MAH.;IDEALTEPE MAH. AYDIN SOKAK 34841 MALTEPE/ISTANBUL
7;ebru.balci67704@outlookerlab.com;Ebru Balcı;1;F;1959-01-31;Marmara;Istanbul;KUCUKCEKMECE;TEVFIK BEY MAH.;TEVFIK BEY MAH. VURAL SOKAK KUCUKCEKMECE/ISTANBUL
8;pelin.yasemin.caglar98652@inboxplus.com;Pelin Yasemin Çağlar;1;F;1958-06-02;Ic Anadolu;Kirikkale;YAHSIHAN;KADIOGLU MAH.;KADIOGLU MAH. KUME EVLER KUME EVLERI 71450 YAHSIHAN/KIRIKKALE
9;gonca.yavuz56286@hotmailia.com;Gonca Yavuz;1;F;1990-05-02;Akdeniz;Antalya;GUNDOGMUS;CALTI KOYU;CALTI KOYU MAH. CALTI MERKEZ MEVKI 07860 GUNDOGMUS/ANTALYA
10;huseyin.taner.ozkanli40122@yahootracker.com;Hüseyin Taner Özkanlı;1;M;1984-06-23;Marmara;Istanbul;SILE;AGVA MERKEZ MAH.;AGVA MERKEZ MAH. OGRETMENLER SOKAK 34990 SILE/ISTANBUL
